In [0]:
import pandas as pd
from zipfile import ZipFile
from urllib.request import urlopen  
import numpy as np
import os

In [0]:
ico_vs = {'00216208': 'Univerzita Karlova',
 '60076658': 'Jihočeská univerzita v Českých Budějovicích',
 '44555601': 'Univerzita J. E. Purkyně v Ústí nad Labem',
 '00216224': 'Masarykova univerzita',
 '61989592': 'Univerzita Palackého v Olomouci',
 '62157124': 'Veterinární a farmaceutická univerzita Brno',
 '61988987': 'Ostravská univerzita',
 '62690094': 'Univerzita  Hradec Králové',
 '47813059': 'Slezská univerzita v Opavě',
 '68407700': 'České vysoké učení technické v Praze',
 '60461373': 'Vysoká škola chemicko-technologická v Praze',
 '49777513': 'Západočeská univerzita v Plzni',
 '46747885': 'Technická univerzita v Liberci',
 '00216275': 'Univerzita Pardubice',
 '00216305': 'Vysoké učení technické v Brně',
 '61989100': 'Vysoká škola báňská - Technická univerzita  Ostrava',
 '70883521': 'Univerzita Tomáše Bati ve Zlíně',
 '61384399': 'Vysoká škola ekonomická v Praze',
 '60460709': 'Česká zemědělská univerzita v Praze',
 '62156489': 'Mendelova univerzita',
 '61384984': 'Akademie múzických umění v Praze',
 '60461446': 'Akademie výtvarných umění v Praze',
 '60461071': 'Vysoká škola umělecko-průmyslová v Praze',
 '62156462': 'Janáčkova akademie múzických umění v Brně'}

# Tabulka č.11-final

In [0]:
# Tabulka č. 11: https://www.mpsv.cz/documents/20142/889529/stat-abs-2018-p2.zip k 30.9.2018

In [0]:
# konstanty 
file_name = 'tabulky2018_p2.zip'
URL = 'https://www.mpsv.cz/documents/20142/889529/stat-abs-2018-p2.zip'

# open and save the zip file onto computer
url = urlopen(URL)
output = open(file_name, 'wb')    # note the flag:  "wb"        
output.write(url.read())
output.close()

In [0]:
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zip: 
    # printing all the contents of the zip file 
    print('Seznam souborů v archivu:')
    zip.printdir() 
  
    # extracting all the files 
    print('\nRozbalování...') 
    zip.extractall() 
    print('Hotovo!')

In [0]:
#výpis pracovního adresáře
!ls -al

In [0]:
#názvy souborů do proměnné zipFiles
zipFiles = ZipFile.namelist(zip)
print(zipFiles)

In [0]:
#načtení jednoho Excelu do DataFrame
# 'zipFiles[2]' znamená třetí položka z listu 'zipFiles'

#df = pd.read_excel('Absolventi podle τkol a oborà_0918.xlsx', skiprows=2, header=None) #sheet_name='Celkem'
df11 = pd.read_excel(zipFiles[0], sheet_name = 'Celkem', skiprows=2, header=None)
df11.columns = ['kod_okresu', 'okres', 'ico', 'izo', 'nazev_fakulta', 'adresa', 'kod_oboru', 'nazev_oboru', 'forma_studia', 'celkem2018', 'divek2018',
              'z Celkem dosazitelni2018', 'z divek dosazitelni2018', 'dosud_nepracujici2018', 'abs_1.5.-30.9.18', 'abs_1.10.17-30.9.18']


In [0]:
#vytvoreni noveho sloupce rid_fakulty, kde RID fakulty je součásti IZO školy (neberou se první tři znaky 800 a poslední znak)
df11['rid_fakulta']=df11['izo'].str[3:8]

# vybrat VVŠ podle číselníku, zobrazí jen VŠ s IČ, které potřebujeme
df11 = df11[df11.ico.isin(ico_vs)]
# přidat názvy VŠ
df11['skola'] = df11.ico.map(ico_vs)
df11.head(3)

,kod_okresu,okres,ico,izo,nazev_fakulta,adresa,kod_oboru,nazev_oboru,forma_studia,celkem2018,divek2018,z Celkem dosazitelni2018,z divek dosazitelni2018,dosud_nepracujici2018,abs_1.5.-30.9.18,abs_1.10.17-30.9.18,rid_fakulta,skola
6,3100,Hlavní město Praha,00216208,,Přírodovědecká fakulta UK Praha,"Albertov č.p. 2038/6, 12800 Praha (okres: Hlav...",16-01-T/000,NaN,DE,1,1,1,1,1,0.0,1.0,,Univerzita Karlova
7,3100,Hlavní město Praha,00216208,,Univerzita Karlova - Lékařská fakulta Hradec K...,"Šimkova č.p. 870/13, 50003 Hradec Králové (okr...",51-03-T/000,NaN,PR,2,1,2,1,2,2.0,2.0,,Univerzita Karlova
8,3100,Hlavní město Praha,00216208,,Filozofická fakulta UK Praha,"náměstí Jana Palacha č.p. 1/2, 11000 Praha (ok...",61-07-T/000,NaN,DE,1,0,1,0,0,0.0,1.0,,Univerzita Karlova


In [0]:
#prázdné buňky (resp. buňky s mezerama) nejdříve převedu na NaN hodnoty, aby se mi pak lépe pracovalo
df11.replace(r'^\s+$', np.nan, regex=True, inplace=True)
df11.head(3)

,kod_okresu,okres,ico,izo,nazev_fakulta,adresa,kod_oboru,nazev_oboru,forma_studia,celkem2018,divek2018,z Celkem dosazitelni2018,z divek dosazitelni2018,dosud_nepracujici2018,abs_1.5.-30.9.18,abs_1.10.17-30.9.18,rid_fakulta,skola
6,3100,Hlavní město Praha,00216208,NaN,Přírodovědecká fakulta UK Praha,"Albertov č.p. 2038/6, 12800 Praha (okres: Hlav...",16-01-T/000,NaN,DE,1,1,1,1,1,0.0,1.0,NaN,Univerzita Karlova
7,3100,Hlavní město Praha,00216208,NaN,Univerzita Karlova - Lékařská fakulta Hradec K...,"Šimkova č.p. 870/13, 50003 Hradec Králové (okr...",51-03-T/000,NaN,PR,2,1,2,1,2,2.0,2.0,NaN,Univerzita Karlova
8,3100,Hlavní město Praha,00216208,NaN,Filozofická fakulta UK Praha,"náměstí Jana Palacha č.p. 1/2, 11000 Praha (ok...",61-07-T/000,NaN,DE,1,0,1,0,0,0.0,1.0,NaN,Univerzita Karlova


In [0]:
#zjišťuju, kolik má každý sloupec NaN hodnot
print(df11.isna().sum())

kod_okresu                    0
okres                         0
ico                           0
izo                         284
nazev_fakulta                 0
adresa                        0
kod_oboru                     0
nazev_oboru                 284
forma_studia                  0
celkem2018                    0
divek2018                     0
z Celkem dosazitelni2018      0
z divek dosazitelni2018       0
dosud_nepracujici2018         0
abs_1.5.-30.9.18              0
abs_1.10.17-30.9.18           0
rid_fakulta                 284
skola                         0
dtype: int64


In [0]:
#doplnění chybějících rid_fakulta (vím, že to vypadá hrozně ten zápis, ale jsem ráda, že mi to funguje, zkoušela jsem pár pokusů načíst ze seznamu isin apod (zkoušela jsem i apply lambda, fillna), 
#ale bohužel pokaždé byl výsledek takový, že v buňce, kde byla hodnota, byl pak nakonec NaN a doplnil jen ty buňky, kde byl předtím NaN) 
#tyhle fakulty jsem vybrala na základě provedené analýzy dat ÚP, které fakulty jsou nejčastěji NEvyplněné (dělala jsem v Excelu), jedná se o zhruba stejné chyby ve všech sledovaných letech 2011-2019
#tady narážím na problém dvou VŠ:
#Univerzita Tomáše Bati ve Zlíně - vznikla z fakulta VUT ve Zlíně, a po celé sledované období od toho roku 2011, tam jsou chybné RID_fakulty, které značí špatnou uviverzitu VUT, 
#a jedná se podle mě a toho co jsem se snažila zjistit o staré RID, navíc od roku 2014 už pak úplně z dat mizí označení fakult úplně a máme k dispozici jen kódy oborů, 
#které se mi nepodařilo vyhledat na webu v nějaké normální podobě, aby se to nemuselo vyhledávat po jednom a dohledávat tak ke každému fakultu zvlášť ručně, 
#asi bych v tomto případě navrhovala tuto Univerzitu pro naše účely z těchto dat vymazat na základě uvedených důvodů
#Univerzita Hradec Králové - jedná se o cca 40 záznamů ke kterým rovněž nemáme fakulty ani IZO jen kody oborů - nevím, zda vymazat jen tyto záznamy, 
#nebo kvůli zkeslení opět celou univerzitu, že ji nebudeme zjišťovat?
df11.loc[df11.nazev_fakulta=='Univerzita Karlova - Lékařská fakulta Hradec Králové', 'rid_fakulta']='11150' 
df11.loc[df11.nazev_fakulta=='Ekonomická fakulta JU České Budějovice', 'rid_fakulta']='12510'
df11.loc[df11.nazev_fakulta=='Filozofická fakulta JU České Budějovice', 'rid_fakulta']='12210'
df11.loc[df11.nazev_fakulta=='Přírodovědecká fakulta JU České Budějovice', 'rid_fakulta']='12310'
df11.loc[df11.nazev_fakulta=='Fakulta sportovních studií MU', 'rid_fakulta']='14510'
df11.loc[df11.nazev_fakulta=='Filozoficko-přírodovědecká fakulta- Slezská univerzita v Opavě', 'rid_fakulta']='19240'
df11.loc[df11.nazev_fakulta=='Fakulta biomedicínského inženýrství ČVUT Praha', 'rid_fakulta']='21460'
df11.loc[df11.nazev_fakulta=='Fakulta biomedicínského inženýrství ČVUT v Praze', 'rid_fakulta']='21460'
df11.loc[df11.nazev_fakulta=='Fakulta informačních technologii ČVUT Praha', 'rid_fakulta']='21240'
df11.loc[df11.nazev_fakulta=='Fakulta zdravotnických studií ZČU Plzeň', 'rid_fakulta']='23310'
df11.loc[df11.nazev_fakulta=='Ústav umění a designu ZČU Plzeň', 'rid_fakulta']='23410'
df11.loc[df11.nazev_fakulta=='Univerzita Pardubice - Fakulta zdravotnických studií', 'rid_fakulta']='25520'
df11.loc[df11.nazev_fakulta=='Univerzita Pardubice - Fakulta elektrotechniky a informatiky', 'rid_fakulta']='25530'
df11.loc[df11.nazev_fakulta=='Fakulta informačních technologií VUT', 'rid_fakulta']='26230'
df11.loc[df11.nazev_fakulta=='Fakulta životního prostředí', 'rid_fakulta']='41330'
df11.loc[df11.nazev_fakulta=='Fakulta životního prostředí České zemědelské univerzity v Praze', 'rid_fakulta']='41330'
df11.loc[df11.nazev_fakulta=='Fakulta regionálního rozvoje a mezinárodních studií MENDELU', 'rid_fakulta']='43310'
df11.loc[df11.nazev_fakulta=='Fakulta lesnická a dřevařská České zemědělské univerzity v Praze', 'rid_fakulta']='41320'     # tady chyběla čárka
df11.loc[df11.nazev_fakulta=='Provozně ekonomická fakulta České zemědělské univerzity v Praze','rid_fakulta']='41110'
df11.loc[df11.nazev_fakulta=='Fakulta agrobiologie, potravinových a přírodních zdrojů České zemědělské univerzity v Praze','rid_fakulta']='41210'

In [0]:
#zjišťuju, kolik má každý sloupec NaN hodnot
print(df11.isna().sum())
df11.head(3)

kod_okresu                    0
okres                         0
ico                           0
izo                         284
nazev_fakulta                 0
adresa                        0
kod_oboru                     0
nazev_oboru                 284
forma_studia                  0
celkem2018                    0
divek2018                     0
z Celkem dosazitelni2018      0
z divek dosazitelni2018       0
dosud_nepracujici2018         0
abs_1.5.-30.9.18              0
abs_1.10.17-30.9.18           0
rid_fakulta                 162
skola                         0
dtype: int64


,kod_okresu,okres,ico,izo,nazev_fakulta,adresa,kod_oboru,nazev_oboru,forma_studia,celkem2018,divek2018,z Celkem dosazitelni2018,z divek dosazitelni2018,dosud_nepracujici2018,abs_1.5.-30.9.18,abs_1.10.17-30.9.18,rid_fakulta,skola
6,3100,Hlavní město Praha,00216208,NaN,Přírodovědecká fakulta UK Praha,"Albertov č.p. 2038/6, 12800 Praha (okres: Hlav...",16-01-T/000,NaN,DE,1,1,1,1,1,0.0,1.0,NaN,Univerzita Karlova
7,3100,Hlavní město Praha,00216208,NaN,Univerzita Karlova - Lékařská fakulta Hradec K...,"Šimkova č.p. 870/13, 50003 Hradec Králové (okr...",51-03-T/000,NaN,PR,2,1,2,1,2,2.0,2.0,11150,Univerzita Karlova
8,3100,Hlavní město Praha,00216208,NaN,Filozofická fakulta UK Praha,"náměstí Jana Palacha č.p. 1/2, 11000 Praha (ok...",61-07-T/000,NaN,DE,1,0,1,0,0,0.0,1.0,NaN,Univerzita Karlova


In [0]:
#seřadím podle rid_fakulty, aby v daším kroku měl vždy výchozí hodnotu pro doplnění rid_fakulty, pokud by byl první řádek prázdný, nemá podle čeho přiřadit rid (pokud jsem to pochopila správně, jak to funguje)
df11.sort_values(by=['rid_fakulta'], ascending=False, inplace=True)
df11.head(3)

,kod_okresu,okres,ico,izo,nazev_fakulta,adresa,kod_oboru,nazev_oboru,forma_studia,celkem2018,divek2018,z Celkem dosazitelni2018,z divek dosazitelni2018,dosud_nepracujici2018,abs_1.5.-30.9.18,abs_1.10.17-30.9.18,rid_fakulta,skola
6246,3802,Frýdek-Místek,62156462,800545303,Divadelní fakulta JAMU Brno,"Mozartova č.p. 647/1, 60200 Brno (okres: Brno-...",82-03-T/000,Dramatická umění - Divadelní dramaturgie,PR,1,1,1,1,1,1.0,1.0,54530,Janáčkova akademie múzických umění v Brně
3364,3602,Hradec Králové,62156462,800545303,Divadelní fakulta JAMU Brno,"Mozartova č.p. 647/1, 60200 Brno (okres: Brno-...",82-03-T/000,Dramatická umění - Divadelní dramaturgie,PR,1,1,1,1,1,1.0,1.0,54530,Janáčkova akademie múzických umění v Brně
4500,3702,Brno-město,62156462,800545303,Divadelní fakulta JAMU Brno,"Mozartova č.p. 647/1, 60200 Brno (okres: Brno-...",82-03-T/000,Dramatická umění - Divadelní dramaturgie,PR,1,0,1,0,0,0.0,0.0,54530,Janáčkova akademie múzických umění v Brně


In [0]:
#doplní chybějící rid fakulty na základě toho, že seskupí stejné fakulty a podle jejich názvu doplní chybějící rid (jedná se o stejné fakulty, které někde IZO (a následně RID) měly a někde ne, 
#tedy doplňuju podle existujícího)
df11['rid_fakulta'] = df11.groupby(['nazev_fakulta'])['rid_fakulta'].ffill()
df11.head(3)

,kod_okresu,okres,ico,izo,nazev_fakulta,adresa,kod_oboru,nazev_oboru,forma_studia,celkem2018,divek2018,z Celkem dosazitelni2018,z divek dosazitelni2018,dosud_nepracujici2018,abs_1.5.-30.9.18,abs_1.10.17-30.9.18,rid_fakulta,skola
6246,3802,Frýdek-Místek,62156462,800545303,Divadelní fakulta JAMU Brno,"Mozartova č.p. 647/1, 60200 Brno (okres: Brno-...",82-03-T/000,Dramatická umění - Divadelní dramaturgie,PR,1,1,1,1,1,1.0,1.0,54530,Janáčkova akademie múzických umění v Brně
3364,3602,Hradec Králové,62156462,800545303,Divadelní fakulta JAMU Brno,"Mozartova č.p. 647/1, 60200 Brno (okres: Brno-...",82-03-T/000,Dramatická umění - Divadelní dramaturgie,PR,1,1,1,1,1,1.0,1.0,54530,Janáčkova akademie múzických umění v Brně
4500,3702,Brno-město,62156462,800545303,Divadelní fakulta JAMU Brno,"Mozartova č.p. 647/1, 60200 Brno (okres: Brno-...",82-03-T/000,Dramatická umění - Divadelní dramaturgie,PR,1,0,1,0,0,0.0,0.0,54530,Janáčkova akademie múzických umění v Brně


In [0]:
#zjišťuju, kolik má každý sloupec NaN hodnot
print(df11.isna().sum())


kod_okresu                    0
okres                         0
ico                           0
izo                         284
nazev_fakulta                 0
adresa                        0
kod_oboru                     0
nazev_oboru                 284
forma_studia                  0
celkem2018                    0
divek2018                     0
z Celkem dosazitelni2018      0
z divek dosazitelni2018       0
dosud_nepracujici2018         0
abs_1.5.-30.9.18              0
abs_1.10.17-30.9.18           0
rid_fakulta                  88
skola                         0
dtype: int64


In [0]:
#u téhle konkrétní tabulky tedy zbývá ještě 88 záznamů, kde nemáme přiřazený RID, většinou to jsou ty dvě VŠ zmíněné výše (UTB Zlín a VÚT) a zbytek tvoří různé ústavy a instituty pod VŠ, 
#které nejsou tedy fakultami a myslím si, že bychom mohli tyto řádky odstranit pro naše účely???

In [0]:
#zahodit řádky, kdy RID=NaN
#df11.drop(df11[df11.rid_fakulta == 'NaN'].index, inplace=True)
 df11.dropna(subset=['rid_fakulta'],inplace=True)

In [0]:
print(df11.isna().sum())

kod_okresu                    0
okres                         0
ico                           0
izo                         196
nazev_fakulta                 0
adresa                        0
kod_oboru                     0
nazev_oboru                 196
forma_studia                  0
celkem2018                    0
divek2018                     0
z Celkem dosazitelni2018      0
z divek dosazitelni2018       0
dosud_nepracujici2018         0
abs_1.5.-30.9.18              0
abs_1.10.17-30.9.18           0
rid_fakulta                   0
skola                         0
dtype: int64


In [0]:
#když jsem si seřadila fakulty dle RID, zjistila jsem ještě, že první dva řádky mají RID=00000, vychází to ze zdrojových dat, kde je tato chyba u IZO fakulty
df11.sort_values(by=['rid_fakulta'], ascending=True, inplace=True)
df11

,kod_okresu,okres,ico,izo,nazev_fakulta,adresa,kod_oboru,nazev_oboru,forma_studia,celkem2018,divek2018,z Celkem dosazitelni2018,z divek dosazitelni2018,dosud_nepracujici2018,abs_1.5.-30.9.18,abs_1.10.17-30.9.18,rid_fakulta,skola
2939,3507,Louny,44555601,000000001,Univerzita Jana Evangelisty Purkyně - Fakulta ...,"Velká Hradební č.p. 13/47, 40001 Ústí nad Labe...",53-41-R/009,Všeobecná sestra,DE,1,1,1,1,1,1.0,1.0,00000,Univerzita J. E. Purkyně v Ústí nad Labem
1151,3210,Praha-západ,44555601,000000001,Univerzita Jana Evangelisty Purkyně - Fakulta ...,"Velká Hradební č.p. 13/47, 40001 Ústí nad Labe...",53-42-R/004,Fyzioterapie,DE,1,1,1,1,0,1.0,1.0,00000,Univerzita J. E. Purkyně v Ústí nad Labem
6396,3803,Karviná,61988987,810001586,Fakulta sociálních studií OU Ostrava,"Fráni Šrámka č.p. 1121/3, 70900 Ostrava (okres...",67-31-R/000,Sociální politika a sociální práce,PR,2,2,2,2,1,1.0,1.0,00158,Ostravská univerzita
5137,3706,Hodonín,61988987,810001586,Fakulta sociálních studií OU Ostrava,"Fráni Šrámka č.p. 1121/3, 70900 Ostrava (okres...",67-31-T/000,Sociální politika a sociální práce,PR,1,1,1,1,1,1.0,1.0,00158,Ostravská univerzita
379,3100,Hlavní město Praha,61988987,810001586,Fakulta sociálních studií OU Ostrava,"Fráni Šrámka č.p. 1121/3, 70900 Ostrava (okres...",53-42-T/017,Koordinace rehabilitace a dlouhodobé zdravotně...,KS,1,1,1,1,1,1.0,1.0,00158,Ostravská univerzita
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453,3201,Benešov,60461071,800539001,VŠUP Praha,"náměstí Jana Palacha č.p. 80/3, 11000 Praha (o...",82-06-T/000,Výtvarná umění,DE,1,0,1,0,1,1.0,1.0,53900,Vysoká škola umělecko-průmyslová v Praze
1591,3304,Pelhřimov,62156462,800545109,Hudební fakulta JAMU Brno,"Komenského náměstí č.p. 609/6, 60200 Brno (okr...",82-01-T/000,Hudební umění - Dirigování orchestru,PR,1,1,1,1,0,1.0,1.0,54510,Janáčkova akademie múzických umění v Brně
4500,3702,Brno-město,62156462,800545303,Divadelní fakulta JAMU Brno,"Mozartova č.p. 647/1, 60200 Brno (okres: Brno-...",82-03-T/000,Dramatická umění - Divadelní dramaturgie,PR,1,0,1,0,0,0.0,0.0,54530,Janáčkova akademie múzických umění v Brně
3364,3602,Hradec Králové,62156462,800545303,Divadelní fakulta JAMU Brno,"Mozartova č.p. 647/1, 60200 Brno (okres: Brno-...",82-03-T/000,Dramatická umění - Divadelní dramaturgie,PR,1,1,1,1,1,1.0,1.0,54530,Janáčkova akademie múzických umění v Brně


In [0]:
#opravuji chybný RID=00000
df11.loc[df11.rid_fakulta=='00000','rid_fakulta']='13450'
df11.head(3)

,kod_okresu,okres,ico,izo,nazev_fakulta,adresa,kod_oboru,nazev_oboru,forma_studia,celkem2018,divek2018,z Celkem dosazitelni2018,z divek dosazitelni2018,dosud_nepracujici2018,abs_1.5.-30.9.18,abs_1.10.17-30.9.18,rid_fakulta,skola
2939,3507,Louny,44555601,000000001,Univerzita Jana Evangelisty Purkyně - Fakulta ...,"Velká Hradební č.p. 13/47, 40001 Ústí nad Labe...",53-41-R/009,Všeobecná sestra,DE,1,1,1,1,1,1.0,1.0,13450,Univerzita J. E. Purkyně v Ústí nad Labem
1151,3210,Praha-západ,44555601,000000001,Univerzita Jana Evangelisty Purkyně - Fakulta ...,"Velká Hradební č.p. 13/47, 40001 Ústí nad Labe...",53-42-R/004,Fyzioterapie,DE,1,1,1,1,0,1.0,1.0,13450,Univerzita J. E. Purkyně v Ústí nad Labem
6396,3803,Karviná,61988987,810001586,Fakulta sociálních studií OU Ostrava,"Fráni Šrámka č.p. 1121/3, 70900 Ostrava (okres...",67-31-R/000,Sociální politika a sociální práce,PR,2,2,2,2,1,1.0,1.0,00158,Ostravská univerzita


In [0]:
#při kontrolním exportu jsem narazila na další chyby v RID fakulta, chyba je u zdrojových dat z ÚP
#opravuji tedy následující RID_fakulta
df11.loc[df11.rid_fakulta=='00158','rid_fakulta']='17200'
df11.loc[df11.rid_fakulta=='00162','rid_fakulta']='19510'
df11.loc[df11.rid_fakulta=='01366','rid_fakulta']='13410'
df11.loc[df11.rid_fakulta=='01367','rid_fakulta']='13440'
df11.loc[df11.rid_fakulta=='03116','rid_fakulta']='31160'
df11.shape

(1683, 18)

In [0]:
#z 5 kategorií formy studia udělám pouze dvě PR (DE=PR) a KS (DA=KS, JI=KS - pouze jeden záznam, pro naše účely to takto zanedbáme)
df11.loc[df11.forma_studia=='DE','forma_studia']='PR'
df11.loc[df11.forma_studia=='DA','forma_studia']='KS'
df11.loc[df11.forma_studia=='JI','forma_studia']='KS'

In [0]:
#odstraňuji řádky se záznamy Univerzity T.B. ve Zlíně - uveden špatný RID, neznáme fakultu, abychom mohli upravit
df11.drop(df11[df11.nazev_fakulta == 'Univerzita Tomáše Bati ve Zlíně'].index, inplace=True)
df11.shape

(1578, 18)

In [0]:
df11.dtypes

kod_okresu                    int64
okres                        object
ico                          object
izo                          object
nazev_fakulta                object
adresa                       object
kod_oboru                    object
nazev_oboru                  object
forma_studia                 object
celkem2018                    int64
divek2018                     int64
z Celkem dosazitelni2018      int64
z divek dosazitelni2018       int64
dosud_nepracujici2018         int64
abs_1.5.-30.9.18            float64
abs_1.10.17-30.9.18         float64
rid_fakulta                  object
skola                        object
dtype: object

In [0]:
#změna typu sloupce 'kod_okresu' na string -- aby se nesčítal, byť číslo, jedná se o kategorickou proměnnou
df11.kod_okresu = df11.kod_okresu.astype(str)

In [0]:
df11.shape

(1578, 18)

In [0]:
#numerické sloupce -- záznamy sečíst
df11g1 = df11.groupby(['rid_fakulta', 'forma_studia']).sum()

In [0]:
#kategorické sloupce
cols = ['kod_okresu', 'okres', 'ico', 'izo', 'nazev_fakulta', 'adresa',
        'forma_studia', 'rid_fakulta', 'skola']

In [0]:
#vzít první záznam
df11g2 = df11[cols].groupby(['rid_fakulta', 'forma_studia']).first()

In [0]:
#zkontrolovat počty řádků u obou datasetů
df11g1.shape

(177, 7)

In [0]:
#počet sloupců je shodný jen náhodou
df11g2.shape

(177, 7)

In [0]:
df11g1.head()

celkem2018  ...  abs_1.10.17-30.9.18
rid_fakulta forma_studia              ...                     
11110       PR                    29  ...                 28.0
11120       PR                     9  ...                  8.0
11130       KS                     1  ...                  1.0
            PR                    10  ...                 10.0
11140       PR                    11  ...                 11.0

[5 rows x 7 columns]

In [0]:
df11g2.head()

kod_okresu  ...               skola
rid_fakulta forma_studia             ...                    
11110       PR                 3508  ...  Univerzita Karlova
11120       PR                 3308  ...  Univerzita Karlova
11130       KS                 3506  ...  Univerzita Karlova
            PR                 3707  ...  Univerzita Karlova
11140       PR                 3212  ...  Univerzita Karlova

[5 rows x 7 columns]

In [0]:
#sloučit oba datasety podle (multi)indexu
df11g = pd.merge(df11g1, df11g2, left_index=True, right_index=True)

In [0]:
#multiindex do sloupců
df11g.reset_index(inplace=True)

In [0]:
#kontrola 7+7+2 
df11g.shape

(177, 16)

In [0]:
df11g.head(12)

,rid_fakulta,forma_studia,celkem2018,divek2018,z Celkem dosazitelni2018,z divek dosazitelni2018,dosud_nepracujici2018,abs_1.5.-30.9.18,abs_1.10.17-30.9.18,kod_okresu,okres,ico,izo,nazev_fakulta,adresa,skola
0,11110,PR,29,25,29,25,24,28.0,28.0,3508,Most,00216208,800111109,1. lékařská fakulta UK Praha,"Kateřinská č.p. 1660/32, 12000 Praha (okres: H...",Univerzita Karlova
1,11120,PR,9,6,9,6,8,8.0,8.0,3308,Tábor,00216208,800111206,3. lékařská fakulta UK Praha,"Ruská č.p. 2411/87, 10000 Praha (okres: Hlavní...",Univerzita Karlova
2,11130,KS,1,1,1,1,0,1.0,1.0,3506,Litoměřice,00216208,800111303,2. lékařská fakulta UK Praha,"V úvalu č.p. 84/1, 15000 Praha (okres: Hlavní ...",Univerzita Karlova
3,11130,PR,10,8,10,8,6,10.0,10.0,3707,Jihlava,00216208,800111303,2. lékařská fakulta UK Praha,"V úvalu č.p. 84/1, 15000 Praha (okres: Hlavní ...",Univerzita Karlova
4,11140,PR,11,10,10,9,9,10.0,11.0,3212,Rakovník,00216208,800111401,UK - lékařská fakulta Plzeň,"Husova č.p. 654/3, 30100 Plzeň (okres: Plzeň-m...",Univerzita Karlova
5,11150,PR,19,13,19,13,14,16.0,16.0,3603,Chrudim,00216208,NaN,Univerzita Karlova - Lékařská fakulta Hradec K...,"Šimkova č.p. 870/13, 50003 Hradec Králové (okr...",Univerzita Karlova
6,11160,PR,3,3,3,3,3,3.0,3.0,3210,Praha-západ,00216208,800111605,Karlova Univerzita - Farmaceutická fakulta Hra...,"Akademika Heyrovského č.p. 1203/8, 50003 Hrade...",Univerzita Karlova
7,11210,PR,13,9,12,8,7,7.0,10.0,3100,Hlavní město Praha,00216208,800112105,Filozofická fakulta UK Praha,"náměstí Jana Palacha č.p. 1/2, 11000 Praha (ok...",Univerzita Karlova
8,11220,PR,7,4,7,4,3,5.0,5.0,3100,Hlavní město Praha,00216208,800112202,Právnická fakulta UK Praha,"náměstí Curieových č.p. 901/7, 11000 Praha (ok...",Univerzita Karlova
9,11230,PR,19,11,19,11,12,14.0,14.0,3100,Hlavní město Praha,00216208,800112300,Fakulta sociálních věd UK Praha,"Smetanovo nábřeží č.p. 995/6, 11000 Praha (okr...",Univerzita Karlova


In [0]:
# nějak moc forem studia??? JI = jiné? , již jsem opravila výše 5 kategorií forem jsem rozdělila do dvou
df11g['forma_studia'].unique()

array(['PR', 'KS'], dtype=object)

In [0]:
len(df11g.rid_fakulta.unique())

133

In [0]:
#počet fakult na školách
df11g.skola.value_counts()

Univerzita Karlova                                     17
Vysoká škola báňská - Technická univerzita  Ostrava    14
Západočeská univerzita v Plzni                         13
Univerzita Palackého v Olomouci                        13
Vysoké učení technické v Brně                          12
Jihočeská univerzita v Českých Budějovicích            11
Masarykova univerzita                                  11
Univerzita Pardubice                                   11
Univerzita J. E. Purkyně v Ústí nad Labem              10
Ostravská univerzita                                   10
České vysoké učení technické v Praze                    9
Technická univerzita v Liberci                          7
Česká zemědělská univerzita v Praze                     7
Mendelova univerzita                                    7
Vysoká škola ekonomická v Praze                         6
Slezská univerzita v Opavě                              5
Vysoká škola chemicko-technologická v Praze             4
Veterinární a 

In [0]:
#počet škol
len(df11g.skola.value_counts())

22

In [0]:
df11g.shape

(177, 16)

In [0]:
#nakonec smazat soubory z archivu 
for zipFile in zipFiles:
  os.remove(zipFile)

In [0]:
# export výsledné tabulky do csv - té poslučované na úrovni formy studia a rid_fakulty
from google.colab import files
df11g.to_csv('nezam2018_sloucene.csv', encoding='1250') 
files.download('nezam2018_sloucene.csv')

In [0]:
# export výsledné tabulky do csv - celkové, zahrnuje i okresy
from google.colab import files
df11.to_csv('nezam2018vse.csv', encoding='1250') 
files.download('nezam2018vse.csv')

# Tabulka č.12-final

In [0]:
# Tabulka č. 12: https://www.mpsv.cz/documents/20142/889529/stat-abs-2017-p2.zip k 30.9.2017

In [0]:
# konstanty 
file_name = 'tabulky2017_p2.zip'
URL = 'https://www.mpsv.cz/documents/20142/889529/stat-abs-2017-p2.zip'

# open and save the zip file onto computer
url = urlopen(URL)
output = open(file_name, 'wb')    # note the flag:  "wb"        
output.write(url.read())
output.close()

In [0]:
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zip: 
    # printing all the contents of the zip file 
    print('Seznam souborů v archivu:')
    zip.printdir() 
  
    # extracting all the files 
    print('\nRozbalování...') 
    zip.extractall() 
    print('Hotovo!')

In [0]:
#výpis pracovního adresáře
!ls -al

In [0]:
#názvy souborů do proměnné zipFiles
zipFiles = ZipFile.namelist(zip)
print(zipFiles)

In [0]:
#načtení jednoho Excelu do DataFrame
# 'zipFiles[2]' znamená třetí položka z listu 'zipFiles'

#df = pd.read_excel('Absolventi podle τkol a oborà_0917.xlsx', skiprows=2, header=None) #sheet_name='Celkem'
df11 = pd.read_excel(zipFiles[0], sheet_name = 'Celkem', skiprows=2, header=None)
df11.columns = ['kod_okresu', 'okres', 'ico', 'izo', 'nazev_fakulta', 'adresa', 'kod_oboru', 'nazev_oboru', 'forma_studia', 'celkem2017', 'divek2017',
              'z Celkem dosazitelni2017', 'z divek dosazitelni2017', 'dosud_nepracujici2017', 'abs_1.5.-30.9.17', 'abs_1.10.16-30.9.17']
df11.head(3)

In [0]:
#vytvoreni noveho sloupce rid_fakulty, kde RID fakulty je součásti IZO školy (neberou se první tři znaky 800 a poslední znak)
df11['rid_fakulta']=df11['izo'].str[3:8]

# vybrat VVŠ podle číselníku, zobrazí jen VŠ s IČ, které potřebujeme
df11 = df11[df11.ico.isin(ico_vs)]
# přidat názvy VŠ
df11['skola'] = df11.ico.map(ico_vs)
df11.head(3)

In [0]:
#prázdné buňky (resp. buňky s mezerama) nejdříve převedu na NaN hodnoty, aby se mi pak lépe pracovalo
df11.replace(r'^\s+$', np.nan, regex=True, inplace=True)
df11.head(3)

In [0]:
#zjišťuju, kolik má každý sloupec NaN hodnot
print(df11.isna().sum())

In [0]:
#doplnění chybějících rid_fakulta (vím, že to vypadá hrozně ten zápis, ale jsem ráda, že mi to funguje, zkoušela jsem pár pokusů načíst ze seznamu isin apod (zkoušela jsem i apply lambda, fillna), 
#ale bohužel pokaždé byl výsledek takový, že v buňce, kde byla hodnota, byl pak nakonec NaN a doplnil jen ty buňky, kde byl předtím NaN) 
#tyhle fakulty jsem vybrala na základě provedené analýzy dat ÚP, které fakulty jsou nejčastěji NEvyplněné (dělala jsem v Excelu), jedná se o zhruba stejné chyby ve všech sledovaných letech 2011-2019
#tady narážím na problém dvou VŠ:
#Univerzita Tomáše Bati ve Zlíně - vznikla z fakulta VUT ve Zlíně, a po celé sledované období od toho roku 2011, tam jsou chybné RID_fakulty, které značí špatnou uviverzitu VUT, 
#a jedná se podle mě a toho co jsem se snažila zjistit o staré RID, navíc od roku 2014 už pak úplně z dat mizí označení fakult úplně a máme k dispozici jen kódy oborů, 
#které se mi nepodařilo vyhledat na webu v nějaké normální podobě, aby se to nemuselo vyhledávat po jednom a dohledávat tak ke každému fakultu zvlášť ručně, 
#asi bych v tomto případě navrhovala tuto Univerzitu pro naše účely z těchto dat vymazat na základě uvedených důvodů
#Univerzita Hradec Králové - jedná se o cca 40 záznamů ke kterým rovněž nemáme fakulty ani IZO jen kody oborů - nevím, zda vymazat jen tyto záznamy, 
#nebo kvůli zkeslení opět celou univerzitu, že ji nebudeme zjišťovat?
df11.loc[df11.nazev_fakulta=='Univerzita Karlova - Lékařská fakulta Hradec Králové', 'rid_fakulta']='11150' 
df11.loc[df11.nazev_fakulta=='Ekonomická fakulta JU České Budějovice', 'rid_fakulta']='12510'
df11.loc[df11.nazev_fakulta=='Filozofická fakulta JU České Budějovice', 'rid_fakulta']='12210'
df11.loc[df11.nazev_fakulta=='Přírodovědecká fakulta JU České Budějovice', 'rid_fakulta']='12310'
df11.loc[df11.nazev_fakulta=='Fakulta sportovních studií MU', 'rid_fakulta']='14510'
df11.loc[df11.nazev_fakulta=='Filozoficko-přírodovědecká fakulta- Slezská univerzita v Opavě', 'rid_fakulta']='19240'
df11.loc[df11.nazev_fakulta=='Fakulta biomedicínského inženýrství ČVUT Praha', 'rid_fakulta']='21460'
df11.loc[df11.nazev_fakulta=='Fakulta biomedicínského inženýrství ČVUT v Praze', 'rid_fakulta']='21460'
df11.loc[df11.nazev_fakulta=='Fakulta informačních technologii ČVUT Praha', 'rid_fakulta']='21240'
df11.loc[df11.nazev_fakulta=='Fakulta zdravotnických studií ZČU Plzeň', 'rid_fakulta']='23310'
df11.loc[df11.nazev_fakulta=='Ústav umění a designu ZČU Plzeň', 'rid_fakulta']='23410'
df11.loc[df11.nazev_fakulta=='Univerzita Pardubice - Fakulta zdravotnických studií', 'rid_fakulta']='25520'
df11.loc[df11.nazev_fakulta=='Univerzita Pardubice - Fakulta elektrotechniky a informatiky', 'rid_fakulta']='25530'
df11.loc[df11.nazev_fakulta=='Fakulta informačních technologií VUT', 'rid_fakulta']='26230'
df11.loc[df11.nazev_fakulta=='Fakulta životního prostředí', 'rid_fakulta']='41330'
df11.loc[df11.nazev_fakulta=='Fakulta životního prostředí České zemědelské univerzity v Praze', 'rid_fakulta']='41330'
df11.loc[df11.nazev_fakulta=='Fakulta regionálního rozvoje a mezinárodních studií MENDELU', 'rid_fakulta']='43310'
df11.loc[df11.nazev_fakulta=='Fakulta lesnická a dřevařská České zemědělské univerzity v Praze', 'rid_fakulta']='41320'     # tady chyběla čárka
df11.loc[df11.nazev_fakulta=='Provozně ekonomická fakulta České zemědělské univerzity v Praze','rid_fakulta']='41110'
df11.loc[df11.nazev_fakulta=='Fakulta agrobiologie, potravinových a přírodních zdrojů České zemědělské univerzity v Praze','rid_fakulta']='41210'

In [0]:
#zjišťuju, kolik má každý sloupec NaN hodnot
print(df11.isna().sum())
df11.head(3)

In [0]:
#seřadím podle rid_fakulty, aby v daším kroku měl vždy výchozí hodnotu pro doplnění rid_fakulty, pokud by byl první řádek prázdný, nemá podle čeho přiřadit rid (pokud jsem to pochopila správně, jak to funguje)
df11.sort_values(by=['rid_fakulta'], ascending=False, inplace=True)
df11.head(3)

In [0]:
#doplní chybějící rid fakulty na základě toho, že seskupí stejné fakulty a podle jejich názvu doplní chybějící rid (jedná se o stejné fakulty, které někde IZO (a následně RID) měly a někde ne, 
#tedy doplňuju podle existujícího)
df11['rid_fakulta'] = df11.groupby(['nazev_fakulta'])['rid_fakulta'].ffill()
df11.head(3)

In [0]:
#zjišťuju, kolik má každý sloupec NaN hodnot
print(df11.isna().sum())


In [0]:
#zahodit řádky, kdy RID=NaN
#df11.drop(df11[df11.rid_fakulta == 'NaN'].index, inplace=True)
 df11.dropna(subset=['rid_fakulta'],inplace=True)

In [0]:
print(df11.isna().sum())

In [0]:
#když jsem si seřadila fakulty dle RID, zjistila jsem ještě, že první dva řádky mají RID=00000, vychází to ze zdrojových dat, kde je tato chyba u IZO fakulty
df11.sort_values(by=['rid_fakulta'], ascending=True, inplace=True)
df11

In [0]:
#opravuji chybný RID=00000
df11.loc[df11.rid_fakulta=='00000','rid_fakulta']='13450'
df11.head(3)

In [0]:
#při kontrolním exportu jsem narazila na další chyby v RID fakulta, chyba je u zdrojových dat z ÚP
#opravuji tedy následující RID_fakulta
df11.loc[df11.rid_fakulta=='00158','rid_fakulta']='17200'
df11.loc[df11.rid_fakulta=='00162','rid_fakulta']='19510'
df11.loc[df11.rid_fakulta=='01366','rid_fakulta']='13410'
df11.loc[df11.rid_fakulta=='01367','rid_fakulta']='13440'
df11.loc[df11.rid_fakulta=='03116','rid_fakulta']='31160'
df11.shape

In [0]:
#z 5 kategorií formy studia udělám pouze dvě PR (DE=PR) a KS (DA=KS, JI=KS - pouze jeden záznam, pro naše účely to takto zanedbáme)
df11.loc[df11.forma_studia=='DE','forma_studia']='PR'
df11.loc[df11.forma_studia=='DA','forma_studia']='KS'
df11.loc[df11.forma_studia=='JI','forma_studia']='KS'
df11.loc[df11.forma_studia=='DI','forma_studia']='KS'

In [0]:
#odstraňuji řádky se záznamy Univerzity T.B. ve Zlíně - uveden špatný RID, neznáme fakultu, abychom mohli upravit
df11.drop(df11[df11.nazev_fakulta == 'Univerzita Tomáše Bati ve Zlíně'].index, inplace=True)
df11.shape

In [0]:
df11.dtypes

In [0]:
#změna typu sloupce 'kod_okresu' na string -- aby se nesčítal, byť číslo, jedná se o kategorickou proměnnou
df11.kod_okresu = df11.kod_okresu.astype(str)

In [0]:
df11.shape

In [0]:
#numerické sloupce -- záznamy sečíst
df11g1 = df11.groupby(['rid_fakulta', 'forma_studia']).sum()

In [0]:
#kategorické sloupce
cols = ['kod_okresu', 'okres', 'ico', 'izo', 'nazev_fakulta', 'adresa',
        'forma_studia', 'rid_fakulta', 'skola']

In [0]:
#vzít první záznam
df11g2 = df11[cols].groupby(['rid_fakulta', 'forma_studia']).first()

In [0]:
#zkontrolovat počty řádků u obou datasetů
df11g1.shape

In [0]:
#počet sloupců je shodný jen náhodou
df11g2.shape

In [0]:
df11g1.head()

In [0]:
df11g2.head()

In [0]:
#sloučit oba datasety podle (multi)indexu
df11g = pd.merge(df11g1, df11g2, left_index=True, right_index=True)

In [0]:
#multiindex do sloupců
df11g.reset_index(inplace=True)

In [0]:
#kontrola 7+7+2 
df11g.shape

In [0]:
df11g.head(12)

In [0]:
# nějak moc forem studia??? JI = jiné? , již jsem opravila výše 5 kategorií forem jsem rozdělila do dvou, tady přibyla další forma DI - přidáno pod KS
df11g['forma_studia'].unique()

In [0]:
len(df11g.rid_fakulta.unique())

In [0]:
#počet fakult na školách
df11g.skola.value_counts()

In [0]:
#počet škol
len(df11g.skola.value_counts())

In [0]:
df11g.shape

In [0]:
#nakonec smazat soubory z archivu 
for zipFile in zipFiles:
  os.remove(zipFile)

In [0]:
# export výsledné tabulky do csv - té poslučované na úrovni formy studia a rid_fakulty
from google.colab import files
df11g.to_csv('nezam2017_sloucene.csv', encoding='1250') 
files.download('nezam2017_sloucene.csv')

In [0]:
# export výsledné tabulky do csv - celkové, zahrnuje i okresy
from google.colab import files
df11.to_csv('nezam2017vse.csv', encoding='1250') 
files.download('nezam2017vse.csv')

# Tabulka č.13-final

In [0]:
# Tabulka č. 13: https://www.mpsv.cz/documents/20142/889529/stat-abs-2016-p2.zip k 30.9.2016

In [0]:
# konstanty 
file_name = 'tabulky2016_p2.zip'
URL = 'https://www.mpsv.cz/documents/20142/889529/stat-abs-2016-p2.zip'

# open and save the zip file onto computer
url = urlopen(URL)
output = open(file_name, 'wb')    # note the flag:  "wb"        
output.write(url.read())
output.close()

In [0]:
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zip: 
    # printing all the contents of the zip file 
    print('Seznam souborů v archivu:')
    zip.printdir() 
  
    # extracting all the files 
    print('\nRozbalování...') 
    zip.extractall() 
    print('Hotovo!')

In [0]:
#výpis pracovního adresáře
!ls -al

In [0]:
#názvy souborů do proměnné zipFiles
zipFiles = ZipFile.namelist(zip)
print(zipFiles)

In [0]:
#načtení jednoho Excelu do DataFrame
# 'zipFiles[2]' znamená třetí položka z listu 'zipFiles'

#df = pd.read_excel('Absolventi podle τkol a oborà_0917.xlsx', skiprows=2, header=None) #sheet_name='Celkem'
df11 = pd.read_excel(zipFiles[0], sheet_name = 'Celkem', skiprows=2, header=None)
df11.columns = ['kod_okresu', 'okres', 'ico', 'izo', 'nazev_fakulta', 'adresa', 'kod_oboru', 'nazev_oboru', 'forma_studia', 'celkem2016', 'divek2016',
              'z Celkem dosazitelni2016', 'z divek dosazitelni2016', 'dosud_nepracujici2016', 'abs_1.5.-30.9.16', 'abs_1.10.15-30.9.16']
df11.head(3)

In [0]:
#vytvoreni noveho sloupce rid_fakulty, kde RID fakulty je součásti IZO školy (neberou se první tři znaky 800 a poslední znak)
df11['rid_fakulta']=df11['izo'].str[3:8]

# vybrat VVŠ podle číselníku, zobrazí jen VŠ s IČ, které potřebujeme
df11 = df11[df11.ico.isin(ico_vs)]
# přidat názvy VŠ
df11['skola'] = df11.ico.map(ico_vs)
df11.head(3)

In [0]:
#prázdné buňky (resp. buňky s mezerama) nejdříve převedu na NaN hodnoty, aby se mi pak lépe pracovalo
df11.replace(r'^\s+$', np.nan, regex=True, inplace=True)
df11.head(3)

In [0]:
#zjišťuju, kolik má každý sloupec NaN hodnot
print(df11.isna().sum())

In [0]:
#doplnění chybějících rid_fakulta (vím, že to vypadá hrozně ten zápis, ale jsem ráda, že mi to funguje, zkoušela jsem pár pokusů načíst ze seznamu isin apod (zkoušela jsem i apply lambda, fillna), 
#ale bohužel pokaždé byl výsledek takový, že v buňce, kde byla hodnota, byl pak nakonec NaN a doplnil jen ty buňky, kde byl předtím NaN) 
#tyhle fakulty jsem vybrala na základě provedené analýzy dat ÚP, které fakulty jsou nejčastěji NEvyplněné (dělala jsem v Excelu), jedná se o zhruba stejné chyby ve všech sledovaných letech 2011-2019
#tady narážím na problém dvou VŠ:
#Univerzita Tomáše Bati ve Zlíně - vznikla z fakulta VUT ve Zlíně, a po celé sledované období od toho roku 2011, tam jsou chybné RID_fakulty, které značí špatnou uviverzitu VUT, 
#a jedná se podle mě a toho co jsem se snažila zjistit o staré RID, navíc od roku 2014 už pak úplně z dat mizí označení fakult úplně a máme k dispozici jen kódy oborů, 
#které se mi nepodařilo vyhledat na webu v nějaké normální podobě, aby se to nemuselo vyhledávat po jednom a dohledávat tak ke každému fakultu zvlášť ručně, 
#asi bych v tomto případě navrhovala tuto Univerzitu pro naše účely z těchto dat vymazat na základě uvedených důvodů
#Univerzita Hradec Králové - jedná se o cca 40 záznamů ke kterým rovněž nemáme fakulty ani IZO jen kody oborů - nevím, zda vymazat jen tyto záznamy, 
#nebo kvůli zkeslení opět celou univerzitu, že ji nebudeme zjišťovat?
df11.loc[df11.nazev_fakulta=='Univerzita Karlova - Lékařská fakulta Hradec Králové', 'rid_fakulta']='11150' 
df11.loc[df11.nazev_fakulta=='Ekonomická fakulta JU České Budějovice', 'rid_fakulta']='12510'
df11.loc[df11.nazev_fakulta=='Filozofická fakulta JU České Budějovice', 'rid_fakulta']='12210'
df11.loc[df11.nazev_fakulta=='Přírodovědecká fakulta JU České Budějovice', 'rid_fakulta']='12310'
df11.loc[df11.nazev_fakulta=='Fakulta sportovních studií MU', 'rid_fakulta']='14510'
df11.loc[df11.nazev_fakulta=='Filozoficko-přírodovědecká fakulta- Slezská univerzita v Opavě', 'rid_fakulta']='19240'
df11.loc[df11.nazev_fakulta=='Fakulta biomedicínského inženýrství ČVUT Praha', 'rid_fakulta']='21460'
df11.loc[df11.nazev_fakulta=='Fakulta biomedicínského inženýrství ČVUT v Praze', 'rid_fakulta']='21460'
df11.loc[df11.nazev_fakulta=='Fakulta informačních technologii ČVUT Praha', 'rid_fakulta']='21240'
df11.loc[df11.nazev_fakulta=='Fakulta zdravotnických studií ZČU Plzeň', 'rid_fakulta']='23310'
df11.loc[df11.nazev_fakulta=='Ústav umění a designu ZČU Plzeň', 'rid_fakulta']='23410'
df11.loc[df11.nazev_fakulta=='Univerzita Pardubice - Fakulta zdravotnických studií', 'rid_fakulta']='25520'
df11.loc[df11.nazev_fakulta=='Univerzita Pardubice - Fakulta elektrotechniky a informatiky', 'rid_fakulta']='25530'
df11.loc[df11.nazev_fakulta=='Fakulta informačních technologií VUT', 'rid_fakulta']='26230'
df11.loc[df11.nazev_fakulta=='Fakulta životního prostředí', 'rid_fakulta']='41330'
df11.loc[df11.nazev_fakulta=='Fakulta životního prostředí České zemědelské univerzity v Praze', 'rid_fakulta']='41330'
df11.loc[df11.nazev_fakulta=='Fakulta regionálního rozvoje a mezinárodních studií MENDELU', 'rid_fakulta']='43310'
df11.loc[df11.nazev_fakulta=='Fakulta lesnická a dřevařská České zemědělské univerzity v Praze', 'rid_fakulta']='41320'     # tady chyběla čárka
df11.loc[df11.nazev_fakulta=='Provozně ekonomická fakulta České zemědělské univerzity v Praze','rid_fakulta']='41110'
df11.loc[df11.nazev_fakulta=='Fakulta agrobiologie, potravinových a přírodních zdrojů České zemědělské univerzity v Praze','rid_fakulta']='41210'

In [0]:
#zjišťuju, kolik má každý sloupec NaN hodnot
print(df11.isna().sum())
df11.head(3)

In [0]:
#seřadím podle rid_fakulty, aby v daším kroku měl vždy výchozí hodnotu pro doplnění rid_fakulty, pokud by byl první řádek prázdný, nemá podle čeho přiřadit rid (pokud jsem to pochopila správně, jak to funguje)
df11.sort_values(by=['rid_fakulta'], ascending=False, inplace=True)
df11.head(3)

In [0]:
#doplní chybějící rid fakulty na základě toho, že seskupí stejné fakulty a podle jejich názvu doplní chybějící rid (jedná se o stejné fakulty, které někde IZO (a následně RID) měly a někde ne, 
#tedy doplňuju podle existujícího)
df11['rid_fakulta'] = df11.groupby(['nazev_fakulta'])['rid_fakulta'].ffill()
df11.head(3)

In [0]:
#zjišťuju, kolik má každý sloupec NaN hodnot
print(df11.isna().sum())


In [0]:
#zahodit řádky, kdy RID=NaN
#df11.drop(df11[df11.rid_fakulta == 'NaN'].index, inplace=True)
 df11.dropna(subset=['rid_fakulta'],inplace=True)

In [0]:
print(df11.isna().sum())

In [0]:
#když jsem si seřadila fakulty dle RID, zjistila jsem ještě, že první dva řádky mají RID=00000, vychází to ze zdrojových dat, kde je tato chyba u IZO fakulty
df11.sort_values(by=['rid_fakulta'], ascending=True, inplace=True)
df11

In [0]:
#opravuji chybný RID=00000
df11.loc[df11.rid_fakulta=='00000','rid_fakulta']='13450'
df11.head(3)

In [0]:
#při kontrolním exportu jsem narazila na další chyby v RID fakulta, chyba je u zdrojových dat z ÚP
#opravuji tedy následující RID_fakulta
df11.loc[df11.rid_fakulta=='00158','rid_fakulta']='17200'
df11.loc[df11.rid_fakulta=='00162','rid_fakulta']='19510'
df11.loc[df11.rid_fakulta=='01366','rid_fakulta']='13410'
df11.loc[df11.rid_fakulta=='01367','rid_fakulta']='13440'
df11.loc[df11.rid_fakulta=='03116','rid_fakulta']='31160'
df11.shape

In [0]:
#z 5 kategorií formy studia udělám pouze dvě PR (DE=PR) a KS (DA=KS, JI=KS - pouze jeden záznam, pro naše účely to takto zanedbáme)
df11.loc[df11.forma_studia=='DE','forma_studia']='PR'
df11.loc[df11.forma_studia=='DA','forma_studia']='KS'
df11.loc[df11.forma_studia=='JI','forma_studia']='KS'
df11.loc[df11.forma_studia=='DI','forma_studia']='KS'

In [0]:
#odstraňuji řádky se záznamy Univerzity T.B. ve Zlíně - uveden špatný RID, neznáme fakultu, abychom mohli upravit
df11.drop(df11[df11.nazev_fakulta == 'Univerzita Tomáše Bati ve Zlíně'].index, inplace=True)
df11.shape

In [0]:
df11.dtypes

In [0]:
#změna typu sloupce 'kod_okresu' na string -- aby se nesčítal, byť číslo, jedná se o kategorickou proměnnou
df11.kod_okresu = df11.kod_okresu.astype(str)

In [0]:
df11.shape

In [0]:
#numerické sloupce -- záznamy sečíst
df11g1 = df11.groupby(['rid_fakulta', 'forma_studia']).sum()

In [0]:
#kategorické sloupce
cols = ['kod_okresu', 'okres', 'ico', 'izo', 'nazev_fakulta', 'adresa',
        'forma_studia', 'rid_fakulta', 'skola']

In [0]:
#vzít první záznam
df11g2 = df11[cols].groupby(['rid_fakulta', 'forma_studia']).first()

In [0]:
#zkontrolovat počty řádků u obou datasetů
df11g1.shape

In [0]:
#počet sloupců je shodný jen náhodou
df11g2.shape

In [0]:
df11g1.head()

In [0]:
df11g2.head()

In [0]:
#sloučit oba datasety podle (multi)indexu
df11g = pd.merge(df11g1, df11g2, left_index=True, right_index=True)

In [0]:
#multiindex do sloupců
df11g.reset_index(inplace=True)

In [0]:
#kontrola 7+7+2 
df11g.shape

In [0]:
df11g.head(12)

In [0]:
# nějak moc forem studia??? JI = jiné? , již jsem opravila výše 5 kategorií forem jsem rozdělila do dvou, tady přibyla další forma DI - přidáno pod KS
df11g['forma_studia'].unique()

In [0]:
len(df11g.rid_fakulta.unique())

In [0]:
#počet fakult na školách
df11g.skola.value_counts()

In [0]:
#počet škol
len(df11g.skola.value_counts())

In [0]:
df11g.shape

In [0]:
#nakonec smazat soubory z archivu 
for zipFile in zipFiles:
  os.remove(zipFile)

In [0]:
# export výsledné tabulky do csv - té poslučované na úrovni formy studia a rid_fakulty
from google.colab import files
df11g.to_csv('nezam2016_sloucene.csv', encoding='1250') 
files.download('nezam2016_sloucene.csv')

In [0]:
# export výsledné tabulky do csv - celkové, zahrnuje i okresy
from google.colab import files
df11.to_csv('nezam2016vse.csv', encoding='1250') 
files.download('nezam2016vse.csv')

# Tabulka č.14-final

In [0]:
# Tabulka č. 14: https://www.mpsv.cz/documents/20142/889529/stat-abs-2015-p2.zip/ k 30.9.2015

In [0]:
# konstanty 
file_name = 'tabulky2015_p2.zip'
URL = 'https://www.mpsv.cz/documents/20142/889529/stat-abs-2015-p2.zip/'

# open and save the zip file onto computer
url = urlopen(URL)
output = open(file_name, 'wb')    # note the flag:  "wb"        
output.write(url.read())
output.close()

In [0]:
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zip: 
    # printing all the contents of the zip file 
    print('Seznam souborů v archivu:')
    zip.printdir() 
  
    # extracting all the files 
    print('\nRozbalování...') 
    zip.extractall() 
    print('Hotovo!')

In [0]:
#výpis pracovního adresáře
!ls -al

In [0]:
#názvy souborů do proměnné zipFiles
zipFiles = ZipFile.namelist(zip)
print(zipFiles)

In [0]:
#načtení jednoho Excelu do DataFrame
# 'zipFiles[2]' znamená třetí položka z listu 'zipFiles'

#df = pd.read_excel('Absolventi podle τkol a oborà_0917.xlsx', skiprows=2, header=None) #sheet_name='Celkem'
df11 = pd.read_excel(zipFiles[0], sheet_name = 'Celkem', skiprows=2, header=None)
df11.columns = ['kod_okresu', 'okres', 'ico', 'izo', 'nazev_fakulta', 'adresa', 'kod_oboru', 'nazev_oboru', 'forma_studia', 'celkem2015', 'divek2015',
              'z Celkem dosazitelni2015', 'z divek dosazitelni2015', 'dosud_nepracujici2015', 'abs_1.5.-30.9.15', 'abs_1.10.15-30.9.15']
df11.head(3)

In [0]:
df11.dtypes

In [0]:
#vytvoreni noveho sloupce rid_fakulty, kde RID fakulty je součásti IZO školy (neberou se první tři znaky 800 a poslední znak)
df11['rid_fakulta']=df11['izo'].str[3:8]

# vybrat VVŠ podle číselníku, zobrazí jen VŠ s IČ, které potřebujeme
df11 = df11[df11.ico.isin(ico_vs)]
# přidat názvy VŠ
df11['skola'] = df11.ico.map(ico_vs)
df11.head(3)

In [0]:
#prázdné buňky (resp. buňky s mezerama) nejdříve převedu na NaN hodnoty, aby se mi pak lépe pracovalo
df11.replace(r'^\s+$', np.nan, regex=True, inplace=True)
df11.head(3)

In [0]:
#zjišťuju, kolik má každý sloupec NaN hodnot
print(df11.isna().sum())

In [0]:
#doplnění chybějících rid_fakulta (vím, že to vypadá hrozně ten zápis, ale jsem ráda, že mi to funguje, zkoušela jsem pár pokusů načíst ze seznamu isin apod (zkoušela jsem i apply lambda, fillna), 
#ale bohužel pokaždé byl výsledek takový, že v buňce, kde byla hodnota, byl pak nakonec NaN a doplnil jen ty buňky, kde byl předtím NaN) 
#tyhle fakulty jsem vybrala na základě provedené analýzy dat ÚP, které fakulty jsou nejčastěji NEvyplněné (dělala jsem v Excelu), jedná se o zhruba stejné chyby ve všech sledovaných letech 2011-2019
#tady narážím na problém dvou VŠ:
#Univerzita Tomáše Bati ve Zlíně - vznikla z fakulta VUT ve Zlíně, a po celé sledované období od toho roku 2011, tam jsou chybné RID_fakulty, které značí špatnou uviverzitu VUT, 
#a jedná se podle mě a toho co jsem se snažila zjistit o staré RID, navíc od roku 2014 už pak úplně z dat mizí označení fakult úplně a máme k dispozici jen kódy oborů, 
#které se mi nepodařilo vyhledat na webu v nějaké normální podobě, aby se to nemuselo vyhledávat po jednom a dohledávat tak ke každému fakultu zvlášť ručně, 
#asi bych v tomto případě navrhovala tuto Univerzitu pro naše účely z těchto dat vymazat na základě uvedených důvodů
#Univerzita Hradec Králové - jedná se o cca 40 záznamů ke kterým rovněž nemáme fakulty ani IZO jen kody oborů - nevím, zda vymazat jen tyto záznamy, 
#nebo kvůli zkeslení opět celou univerzitu, že ji nebudeme zjišťovat?
df11.loc[df11.nazev_fakulta=='Univerzita Karlova - Lékařská fakulta Hradec Králové', 'rid_fakulta']='11150' 
df11.loc[df11.nazev_fakulta=='Ekonomická fakulta JU České Budějovice', 'rid_fakulta']='12510'
df11.loc[df11.nazev_fakulta=='Filozofická fakulta JU České Budějovice', 'rid_fakulta']='12210'
df11.loc[df11.nazev_fakulta=='Přírodovědecká fakulta JU České Budějovice', 'rid_fakulta']='12310'
df11.loc[df11.nazev_fakulta=='Fakulta sportovních studií MU', 'rid_fakulta']='14510'
df11.loc[df11.nazev_fakulta=='Filozoficko-přírodovědecká fakulta- Slezská univerzita v Opavě', 'rid_fakulta']='19240'
df11.loc[df11.nazev_fakulta=='Fakulta biomedicínského inženýrství ČVUT Praha', 'rid_fakulta']='21460'
df11.loc[df11.nazev_fakulta=='Fakulta biomedicínského inženýrství ČVUT v Praze', 'rid_fakulta']='21460'
df11.loc[df11.nazev_fakulta=='Fakulta informačních technologii ČVUT Praha', 'rid_fakulta']='21240'
df11.loc[df11.nazev_fakulta=='Fakulta zdravotnických studií ZČU Plzeň', 'rid_fakulta']='23310'
df11.loc[df11.nazev_fakulta=='Ústav umění a designu ZČU Plzeň', 'rid_fakulta']='23410'
df11.loc[df11.nazev_fakulta=='Univerzita Pardubice - Fakulta zdravotnických studií', 'rid_fakulta']='25520'
df11.loc[df11.nazev_fakulta=='Univerzita Pardubice - Fakulta elektrotechniky a informatiky', 'rid_fakulta']='25530'
df11.loc[df11.nazev_fakulta=='Fakulta informačních technologií VUT', 'rid_fakulta']='26230'
df11.loc[df11.nazev_fakulta=='Fakulta životního prostředí', 'rid_fakulta']='41330'
df11.loc[df11.nazev_fakulta=='Fakulta životního prostředí České zemědelské univerzity v Praze', 'rid_fakulta']='41330'
df11.loc[df11.nazev_fakulta=='Fakulta regionálního rozvoje a mezinárodních studií MENDELU', 'rid_fakulta']='43310'
df11.loc[df11.nazev_fakulta=='Fakulta lesnická a dřevařská České zemědělské univerzity v Praze', 'rid_fakulta']='41320'     # tady chyběla čárka
df11.loc[df11.nazev_fakulta=='Provozně ekonomická fakulta České zemědělské univerzity v Praze','rid_fakulta']='41110'
df11.loc[df11.nazev_fakulta=='Fakulta agrobiologie, potravinových a přírodních zdrojů České zemědělské univerzity v Praze','rid_fakulta']='41210'

In [0]:
#zjišťuju, kolik má každý sloupec NaN hodnot
print(df11.isna().sum())
df11.head(3)

In [0]:
#seřadím podle rid_fakulty, aby v daším kroku měl vždy výchozí hodnotu pro doplnění rid_fakulty, pokud by byl první řádek prázdný, nemá podle čeho přiřadit rid (pokud jsem to pochopila správně, jak to funguje)
df11.sort_values(by=['rid_fakulta'], ascending=False, inplace=True)
df11.head(3)

In [0]:
#doplní chybějící rid fakulty na základě toho, že seskupí stejné fakulty a podle jejich názvu doplní chybějící rid (jedná se o stejné fakulty, které někde IZO (a následně RID) měly a někde ne, 
#tedy doplňuju podle existujícího)
df11['rid_fakulta'] = df11.groupby(['nazev_fakulta'])['rid_fakulta'].ffill()
df11.head(3)

In [0]:
#zjišťuju, kolik má každý sloupec NaN hodnot
print(df11.isna().sum())


In [0]:
#zahodit řádky, kdy RID=NaN
#df11.drop(df11[df11.rid_fakulta == 'NaN'].index, inplace=True)
 df11.dropna(subset=['rid_fakulta'],inplace=True)

In [0]:
print(df11.isna().sum())

In [0]:
#když jsem si seřadila fakulty dle RID, zjistila jsem ještě, že první dva řádky mají RID=00000, vychází to ze zdrojových dat, kde je tato chyba u IZO fakulty
df11.sort_values(by=['rid_fakulta'], ascending=True, inplace=True)
df11

In [0]:
#opravuji chybný RID=00000
df11.loc[df11.rid_fakulta=='00000','rid_fakulta']='13450'
df11.head(3)

In [0]:
#při kontrolním exportu jsem narazila na další chyby v RID fakulta, chyba je u zdrojových dat z ÚP
#opravuji tedy následující RID_fakulta
df11.loc[df11.rid_fakulta=='00158','rid_fakulta']='17200'
df11.loc[df11.rid_fakulta=='00162','rid_fakulta']='19510'
df11.loc[df11.rid_fakulta=='01366','rid_fakulta']='13410'
df11.loc[df11.rid_fakulta=='01367','rid_fakulta']='13440'
df11.loc[df11.rid_fakulta=='03116','rid_fakulta']='31160'
df11.shape

In [0]:
#z 5 kategorií formy studia udělám pouze dvě PR (DE=PR) a KS (DA=KS, JI=KS - pouze jeden záznam, pro naše účely to takto zanedbáme)
df11.loc[df11.forma_studia=='DE','forma_studia']='PR'
df11.loc[df11.forma_studia=='DA','forma_studia']='KS'
df11.loc[df11.forma_studia=='JI','forma_studia']='KS'
df11.loc[df11.forma_studia=='DI','forma_studia']='KS'

In [0]:
#odstraňuji řádky se záznamy Univerzity T.B. ve Zlíně - uveden špatný RID, neznáme fakultu, abychom mohli upravit
df11.drop(df11[df11.nazev_fakulta == 'Univerzita Tomáše Bati ve Zlíně'].index, inplace=True)
df11.shape

In [0]:
df11.dtypes

In [0]:
#změna typu sloupce 'kod_okresu' na string -- aby se nesčítal, byť číslo, jedná se o kategorickou proměnnou
df11.kod_okresu = df11.kod_okresu.astype(str)

In [0]:
df11.shape

In [0]:
#numerické sloupce -- záznamy sečíst
df11g1 = df11.groupby(['rid_fakulta', 'forma_studia']).sum()

In [0]:
#kategorické sloupce
cols = ['kod_okresu', 'okres', 'ico', 'izo', 'nazev_fakulta', 'adresa',
        'forma_studia', 'rid_fakulta', 'skola']

In [0]:
#vzít první záznam
df11g2 = df11[cols].groupby(['rid_fakulta', 'forma_studia']).first()

In [0]:
#zkontrolovat počty řádků u obou datasetů
df11g1.shape

In [0]:
#počet sloupců je shodný jen náhodou
df11g2.shape

In [0]:
df11g1.head()

In [0]:
df11g2.head()

In [0]:
#sloučit oba datasety podle (multi)indexu
df11g = pd.merge(df11g1, df11g2, left_index=True, right_index=True)

In [0]:
#multiindex do sloupců
df11g.reset_index(inplace=True)

In [0]:
#kontrola 7+7+2 
df11g.shape

In [0]:
df11g.head(12)

In [0]:
# nějak moc forem studia??? JI = jiné? , již jsem opravila výše 5 kategorií forem jsem rozdělila do dvou, tady přibyla další forma DI - přidáno pod KS
df11g['forma_studia'].unique()

In [0]:
len(df11g.rid_fakulta.unique())

In [0]:
#počet fakult na školách
df11g.skola.value_counts()

In [0]:
#počet škol
len(df11g.skola.value_counts())

In [0]:
df11g.shape

In [0]:
#nakonec smazat soubory z archivu 
for zipFile in zipFiles:
  os.remove(zipFile)

In [0]:
# export výsledné tabulky do csv - té poslučované na úrovni formy studia a rid_fakulty
from google.colab import files
df11g.to_csv('nezam2015_sloucene.csv', encoding='1250') 
files.download('nezam2015_sloucene.csv')

In [0]:
# export výsledné tabulky do csv - celkové, zahrnuje i okresy
from google.colab import files
df11.to_csv('nezam2015vse.csv', encoding='1250') 
files.download('nezam2015vse.csv')

# Tabulka č.15-final

In [0]:
# Tabulka č. 15: https://www.mpsv.cz/documents/20142/889529/stat-abs-2014-p2.zip/ k 30.9.2014

In [0]:
# konstanty 
file_name = 'tabulky2014_p2.zip'
URL = 'https://www.mpsv.cz/documents/20142/889529/stat-abs-2014-p2.zip/'

# open and save the zip file onto computer
url = urlopen(URL)
output = open(file_name, 'wb')    # note the flag:  "wb"        
output.write(url.read())
output.close()

In [0]:
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zip: 
    # printing all the contents of the zip file 
    print('Seznam souborů v archivu:')
    zip.printdir() 
  
    # extracting all the files 
    print('\nRozbalování...') 
    zip.extractall() 
    print('Hotovo!')

In [0]:
#výpis pracovního adresáře
!ls -al

In [0]:
#názvy souborů do proměnné zipFiles
zipFiles = ZipFile.namelist(zip)
print(zipFiles)

In [0]:
#načtení jednoho Excelu do DataFrame
# 'zipFiles[2]' znamená třetí položka z listu 'zipFiles'

#df = pd.read_excel('Absolventi podle τkol a oborà_0917.xlsx', skiprows=2, header=None) #sheet_name='Celkem'
df11 = pd.read_excel(zipFiles[0], sheet_name = 'Celkem', skiprows=2, header=None)
df11.columns = ['kod_okresu', 'okres', 'ico', 'izo', 'nazev_fakulta', 'adresa', 'kod_oboru', 'nazev_oboru', 'forma_studia', 'celkem2014', 'divek2014',
              'z Celkem dosazitelni2014', 'z divek dosazitelni2014', 'dosud_nepracujici2014', 'abs_1.5.-30.9.14', 'abs_1.10.15-30.9.14']
df11.head(3)
df11.shape

In [0]:
df11.dtypes

In [0]:
#vytvoreni noveho sloupce rid_fakulty, kde RID fakulty je součásti IZO školy (neberou se první tři znaky 800 a poslední znak)
df11['rid_fakulta']=df11['izo'].str[3:8]

# vybrat VVŠ podle číselníku, zobrazí jen VŠ s IČ, které potřebujeme
df11 = df11[df11.ico.isin(ico_vs)]
# přidat názvy VŠ
df11['skola'] = df11.ico.map(ico_vs)
df11.head(3)
df11.shape

In [0]:
#prázdné buňky (resp. buňky s mezerama) nejdříve převedu na NaN hodnoty, aby se mi pak lépe pracovalo
df11.replace(r'^\s+$', np.nan, regex=True, inplace=True)
df11.head(3)

In [0]:
#zjišťuju, kolik má každý sloupec NaN hodnot
print(df11.isna().sum())

In [0]:
#doplnění chybějících rid_fakulta 
df11.loc[df11.nazev_fakulta=='Univerzita Karlova - Lékařská fakulta Hradec Králové', 'rid_fakulta']='11150' 
df11.loc[df11.nazev_fakulta=='Ekonomická fakulta JU České Budějovice', 'rid_fakulta']='12510'
df11.loc[df11.nazev_fakulta=='Filozofická fakulta JU České Budějovice', 'rid_fakulta']='12210'
df11.loc[df11.nazev_fakulta=='Přírodovědecká fakulta JU České Budějovice', 'rid_fakulta']='12310'
df11.loc[df11.nazev_fakulta=='Fakulta sportovních studií MU', 'rid_fakulta']='14510'
df11.loc[df11.nazev_fakulta=='Filozoficko-přírodovědecká fakulta- Slezská univerzita v Opavě', 'rid_fakulta']='19240'
df11.loc[df11.nazev_fakulta=='Fakulta biomedicínského inženýrství ČVUT Praha', 'rid_fakulta']='21460'
df11.loc[df11.nazev_fakulta=='Fakulta biomedicínského inženýrství ČVUT v Praze', 'rid_fakulta']='21460'
df11.loc[df11.nazev_fakulta=='Fakulta informačních technologii ČVUT Praha', 'rid_fakulta']='21240'
df11.loc[df11.nazev_fakulta=='Fakulta zdravotnických studií ZČU Plzeň', 'rid_fakulta']='23310'
df11.loc[df11.nazev_fakulta=='Ústav umění a designu ZČU Plzeň', 'rid_fakulta']='23410'
df11.loc[df11.nazev_fakulta=='Univerzita Pardubice - Fakulta zdravotnických studií', 'rid_fakulta']='25520'
df11.loc[df11.nazev_fakulta=='Univerzita Pardubice - Fakulta elektrotechniky a informatiky', 'rid_fakulta']='25530'
df11.loc[df11.nazev_fakulta=='Fakulta informačních technologií VUT', 'rid_fakulta']='26230'
df11.loc[df11.nazev_fakulta=='Fakulta životního prostředí', 'rid_fakulta']='41330'
df11.loc[df11.nazev_fakulta=='Fakulta životního prostředí České zemědelské univerzity v Praze', 'rid_fakulta']='41330'
df11.loc[df11.nazev_fakulta=='Fakulta regionálního rozvoje a mezinárodních studií MENDELU', 'rid_fakulta']='43310'
df11.loc[df11.nazev_fakulta=='Fakulta lesnická a dřevařská České zemědělské univerzity v Praze', 'rid_fakulta']='41320'     # tady chyběla čárka
df11.loc[df11.nazev_fakulta=='Provozně ekonomická fakulta České zemědělské univerzity v Praze','rid_fakulta']='41110'
df11.loc[df11.nazev_fakulta=='Fakulta agrobiologie, potravinových a přírodních zdrojů České zemědělské univerzity v Praze','rid_fakulta']='41210'

In [0]:
#zjišťuju, kolik má každý sloupec NaN hodnot
print(df11.isna().sum())
df11.head(3)

In [0]:
#seřadím podle rid_fakulty, aby v daším kroku měl vždy výchozí hodnotu pro doplnění rid_fakulty, pokud by byl první řádek prázdný, nemá podle čeho přiřadit rid (pokud jsem to pochopila správně, jak to funguje)
df11.sort_values(by=['rid_fakulta'], ascending=False, inplace=True)
df11.head(3)

In [0]:
#doplní chybějící rid fakulty na základě toho, že seskupí stejné fakulty a podle jejich názvu doplní chybějící rid (jedná se o stejné fakulty, které někde IZO (a následně RID) měly a někde ne, 
#tedy doplňuju podle existujícího)
df11['rid_fakulta'] = df11.groupby(['nazev_fakulta'])['rid_fakulta'].ffill()
df11.head(3)

In [0]:
#zjišťuju, kolik má každý sloupec NaN hodnot
print(df11.isna().sum())


In [0]:
#zahodit řádky, kdy RID=NaN
#df11.drop(df11[df11.rid_fakulta == 'NaN'].index, inplace=True)
 df11.dropna(subset=['rid_fakulta'],inplace=True)

In [0]:
print(df11.isna().sum())

In [0]:
#když jsem si seřadila fakulty dle RID, zjistila jsem ještě, že první dva řádky mají RID=00000, vychází to ze zdrojových dat, kde je tato chyba u IZO fakulty
df11.sort_values(by=['rid_fakulta'], ascending=True, inplace=True)
df11

In [0]:
#opravuji chybný RID=00000
df11.loc[df11.rid_fakulta=='00000','rid_fakulta']='13450'
df11.head(3)

In [0]:
#při kontrolním exportu jsem narazila na další chyby v RID fakulta, chyba je u zdrojových dat z ÚP
#opravuji tedy následující RID_fakulta
df11.loc[df11.rid_fakulta=='00158','rid_fakulta']='17200'
df11.loc[df11.rid_fakulta=='00162','rid_fakulta']='19510'
df11.loc[df11.rid_fakulta=='01366','rid_fakulta']='13410'
df11.loc[df11.rid_fakulta=='01367','rid_fakulta']='13440'
df11.loc[df11.rid_fakulta=='03116','rid_fakulta']='31160'
df11.shape

In [0]:
#z 5 kategorií formy studia udělám pouze dvě PR (DE=PR) a KS (DA=KS, JI=KS - pouze jeden záznam, pro naše účely to takto zanedbáme)
df11.loc[df11.forma_studia=='DE','forma_studia']='PR'
df11.loc[df11.forma_studia=='DA','forma_studia']='KS'
df11.loc[df11.forma_studia=='JI','forma_studia']='KS'
df11.loc[df11.forma_studia=='DI','forma_studia']='KS'

In [0]:
#odstraňuji řádky se záznamy Univerzity T.B. ve Zlíně - uveden špatný RID, neznáme fakultu, abychom mohli upravit
df11.drop(df11[df11.nazev_fakulta == 'Univerzita Tomáše Bati ve Zlíně'].index, inplace=True)
df11.shape

In [0]:
df11.dtypes

In [0]:
#změna typu sloupce 'kod_okresu' na string -- aby se nesčítal, byť číslo, jedná se o kategorickou proměnnou
df11.kod_okresu = df11.kod_okresu.astype(str)

In [0]:
df11.shape

In [0]:
#numerické sloupce -- záznamy sečíst
df11g1 = df11.groupby(['rid_fakulta', 'forma_studia']).sum()

In [0]:
#kategorické sloupce
cols = ['kod_okresu', 'okres', 'ico', 'izo', 'nazev_fakulta', 'adresa',
        'forma_studia', 'rid_fakulta', 'skola']

In [0]:
#vzít první záznam
df11g2 = df11[cols].groupby(['rid_fakulta', 'forma_studia']).first()

In [0]:
#zkontrolovat počty řádků u obou datasetů
df11g1.shape

In [0]:
#počet sloupců je shodný jen náhodou
df11g2.shape

In [0]:
df11g1.head()

In [0]:
df11g2.head()

In [0]:
#sloučit oba datasety podle (multi)indexu
df11g = pd.merge(df11g1, df11g2, left_index=True, right_index=True)

In [0]:
#multiindex do sloupců
df11g.reset_index(inplace=True)

In [0]:
#kontrola 7+7+2 
df11g.shape

In [0]:
df11g.head(12)

In [0]:
# nějak moc forem studia??? JI = jiné? , již jsem opravila výše 5 kategorií forem jsem rozdělila do dvou, tady přibyla další forma DI - přidáno pod KS
df11g['forma_studia'].unique()

In [0]:
len(df11g.rid_fakulta.unique())

In [0]:
#počet fakult na školách
df11g.skola.value_counts()

In [0]:
#počet škol
len(df11g.skola.value_counts())

In [0]:
df11g.shape

In [0]:
#nakonec smazat soubory z archivu 
for zipFile in zipFiles:
  os.remove(zipFile)

In [0]:
# export výsledné tabulky do csv - té poslučované na úrovni formy studia a rid_fakulty
from google.colab import files
df11g.to_csv('nezam2014_sloucene.csv', encoding='1250') 
files.download('nezam2014_sloucene.csv')

In [0]:
# export výsledné tabulky do csv - celkové, zahrnuje i okresy
from google.colab import files
df11.to_csv('nezam2014vse.csv', encoding='1250') 
files.download('nezam2014vse.csv')